In [1]:
import pyspark

In [11]:
!pip install pyspark --quiet


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [97]:
from pyspark import SparkContext as sc

In [98]:
from pyspark.sql import SparkSession

In [99]:
import pandas as pd
import numpy as np

In [100]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [152]:
from pyspark.ml import pipeline

In [102]:
from pyspark.ml.feature import StandardScaler,StringIndexer, VectorAssembler, VectorIndexer, OneHotEncoder

In [8]:
import sklearn
from sklearn.model_selection import train_test_split

In [9]:
import gc

In [103]:
spark = (SparkSession.builder
                  .getOrCreate())

In [11]:
spark.version

'3.2.1'

In [104]:
flights = spark.read.format("csv") \
       .option("header", "true") \
       .load('../Datasets/flights.csv') 

In [105]:
flights.cache

<bound method DataFrame.cache of DataFrame[year: string, month: string, day: string, dep_time: string, dep_delay: string, arr_time: string, arr_delay: string, carrier: string, tailnum: string, flight: string, origin: string, dest: string, air_time: string, distance: string, hour: string, minute: string]>

In [114]:
flights.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



In [109]:
flights = flights.withColumn('air_time', flights.air_time.cast('int'))

In [111]:
flights = flights.withColumn('distance', flights.distance.cast('int'))

In [113]:
flights = flights.withColumn('dep_delay', flights.dep_delay.cast('int'))

In [ ]:
flights = flights.withColumn('air_time', flights.air_time.cast('int'))

In [117]:
flights = flights.withColumn('arr_delay', flights.arr_delay.cast('int'))

In [15]:
print(spark)

In [16]:
print(spark.catalog.listTables())

[]


In [17]:
flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [18]:
flights.select('month','day').show(5)

+-----+---+
|month|day|
+-----+---+
|   12|  8|
|    1| 22|
|    3|  9|
|    4|  9|
|    3|  9|
+-----+---+
only showing top 5 rows



In [115]:
spark.sql('SELECT month, day FROM flights')

AnalysisException: Table or view not found: flights; line 1 pos 23;
'Project ['month, 'day]
+- 'UnresolvedRelation [flights], [], false


In [20]:
sea = flights.select('month', 'day', 'origin', 'dest').where("origin == 'SEA'")

In [21]:
pd_sea= sea.toPandas()

In [22]:
pd_sea.head()

,month,day,origin,dest
0,12,8,SEA,LAX
1,1,22,SEA,HNL
2,3,9,SEA,SFO
3,3,9,SEA,BUR
4,5,12,SEA,SFO


In [23]:
pdx= flights.select('month', 'day', 'origin', 'dest', 'arr_time').where("origin == 'PDX'")

In [24]:
pdx_pd= pdx.toPandas()

In [25]:
pdx_pd.head()

,month,day,origin,dest,arr_time
0,4,9,PDX,SJC,1839
1,1,15,PDX,DEN,1352
2,7,2,PDX,OAK,1041
3,1,29,PDX,SFO,2159
4,6,5,PDX,BUR,1945


In [26]:
pd_temp = pd.DataFrame(np.random.random(10))

In [27]:
spark_temp = spark.createDataFrame(pd_temp)

In [28]:
spark.catalog.listTables()

[]

In [29]:
spark_temp.createOrReplaceTempView('temp')

In [30]:
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [89]:
airports = spark.read.csv('../Datasets/airports.csv', header=True)

In [90]:
airports.show(5)

+--------+--------------------+----------+-----------+-------+------+--------+
|"""faa""|            ""name""|   ""lat""|    ""lon""|""alt""|""tz""|""dst"""|
+--------+--------------------+----------+-----------+-------+------+--------+
|"""04G""|""Lansdowne Airpo...|41.1304722|-80.6195833|   1044|    -5|  ""A"""|
|"""06A""|""Moton Field Mun...|32.4605722|-85.6800278|    264|    -5|  ""A"""|
|"""06C""|""Schaumburg Regi...|41.9893408|-88.1012428|    801|    -6|  ""A"""|
|"""06N""| ""Randall Airport""| 41.431912|-74.3915611|    523|    -5|  ""A"""|
|"""09J""|""Jekyll Island A...|31.0744722|-81.4277778|     11|    -4|  ""A"""|
+--------+--------------------+----------+-----------+-------+------+--------+
only showing top 5 rows



In [33]:
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [34]:
flights.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 2 rows



In [35]:
flights = flights.withColumn('arr_time_hrs', flights.arr_time / 60)

In [36]:
flights.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      arr_time_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|15.583333333333334|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|25.083333333333332|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
only showing top 2 rows



In [37]:
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [38]:
hrs1 = flights.filter('arr_time_hrs > 20')

In [39]:
hrs1.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      arr_time_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|25.083333333333332|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|27.533333333333335|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|             30.65|
|2014|    1| 15|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|  10|    37|22.533333333333335|
|2014|    5| 12|    1655|  

In [40]:
flights.filter('origin == "PDX"').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      arr_time_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|             30.65|
|2014|    1| 15|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|  10|    37|22.533333333333335|
|2014|    7|  2|     847|       42|    1041|       51|     WN| N422WN|  1520|   PDX| OAK|      90|     543|   8|    47|             17.35|
|2014|    1| 29|    2009|        3|    2159|        9|     UA| N27205|  1458|   PDX| SFO|      90|     550|  20|     9|35.983333333333334|
|2014|    6|  5|    1733|  

In [41]:
flights.filter('dest == "LAX"').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      arr_time_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|15.583333333333334|
|2014|   11|  8|    1653|       -2|    1924|       -1|     AS| N323AS|   448|   SEA| LAX|     130|     954|  16|    53| 32.06666666666667|
|2014|    8| 19|    1845|       -5|    2112|      -22|     DL| N354NW|  2642|   SEA| LAX|     119|     954|  18|    45|              35.2|
|2014|    6| 11|     750|      -10|    1009|      -11|     AS| N799AS|   568|   PDX| LAX|     118|     834|   7|    50|16.816666666666666|
|2014|    4| 25|    1049|  

In [42]:
flights.filter('dest == "JFK"').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      arr_time_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12| 26|    2337|        0|     741|       -1|     DL| N3760C|  1358|   PDX| JFK|     274|    2454|  23|    37|             12.35|
|2014|    7| 28|    1047|       -8|    1921|       11|     DL| N713TW|  1473|   SEA| JFK|     269|    2422|  10|    47|32.016666666666666|
|2014|    8| 18|    2116|       -4|     541|       18|     B6| N536JB|   264|   SEA| JFK|     304|    2422|  21|    16| 9.016666666666667|
|2014|    6| 22|    1233|       -7|    2049|      -11|     B6| N632JB|   464|   SEA| JFK|     297|    2422|  12|    33|             34.15|
|2014|    7|  1|     709|  

In [44]:
flights.arr_delay= flights.withColumn('arr_delay', flights.arr_delay.cast('int'))

In [45]:
flights.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- arr_time_hrs: double (nullable = true)



In [46]:
df2= flights.select(flights.origin, flights.dest, flights.arr_time, flights.arr_time_hrs, flights.year, flights.month)

In [47]:
df2.show()

+------+----+--------+------------------+----+-----+
|origin|dest|arr_time|      arr_time_hrs|year|month|
+------+----+--------+------------------+----+-----+
|   SEA| LAX|     935|15.583333333333334|2014|   12|
|   SEA| HNL|    1505|25.083333333333332|2014|    1|
|   SEA| SFO|    1652|27.533333333333335|2014|    3|
|   PDX| SJC|    1839|             30.65|2014|    4|
|   SEA| BUR|    1015|16.916666666666668|2014|    3|
|   PDX| DEN|    1352|22.533333333333335|2014|    1|
|   PDX| OAK|    1041|             17.35|2014|    7|
|   SEA| SFO|    1842|              30.7|2014|    5|
|   SEA| SAN|    1508|25.133333333333333|2014|    4|
|   SEA| ORD|    2352|              39.2|2014|   11|
|   SEA| LAX|    1924| 32.06666666666667|2014|   11|
|   SEA| PHX|    1415|23.583333333333332|2014|    8|
|   SEA| LAS|    1038|              17.3|2014|   10|
|   SEA| ANC|     217|3.6166666666666667|2014|   11|
|   SEA| SFO|    1544|25.733333333333334|2014|   10|
|   PDX| SFO|    2159|35.983333333333334|2014|

In [48]:
df2.filter('origin == "SEA"').show(2)

+------+----+--------+------------------+----+-----+
|origin|dest|arr_time|      arr_time_hrs|year|month|
+------+----+--------+------------------+----+-----+
|   SEA| LAX|     935|15.583333333333334|2014|   12|
|   SEA| HNL|    1505|25.083333333333332|2014|    1|
+------+----+--------+------------------+----+-----+
only showing top 2 rows



In [49]:
df2.filter('origin == "SEA"').filter('dest == "MIA"').show()

+------+----+--------+------------------+----+-----+
|origin|dest|arr_time|      arr_time_hrs|year|month|
+------+----+--------+------------------+----+-----+
|   SEA| MIA|     644|10.733333333333333|2014|    3|
|   SEA| MIA|     745|12.416666666666666|2014|    1|
|   SEA| MIA|     811|13.516666666666667|2014|    9|
|   SEA| MIA|     929|15.483333333333333|2014|   11|
|   SEA| MIA|     809|13.483333333333333|2014|    9|
|   SEA| MIA|     653|10.883333333333333|2014|    3|
|   SEA| MIA|     638|10.633333333333333|2014|    3|
|   SEA| MIA|     541| 9.016666666666667|2014|    6|
|   SEA| MIA|     746|12.433333333333334|2014|    3|
|   SEA| MIA|     608|10.133333333333333|2014|    8|
|   SEA| MIA|     645|             10.75|2014|    4|
|   SEA| MIA|     633|             10.55|2014|    2|
|   SEA| MIA|     824|13.733333333333333|2014|   11|
|   SEA| MIA|     529| 8.816666666666666|2014|    5|
|   SEA| MIA|     630|              10.5|2014|    1|
|   SEA| MIA|     653|10.883333333333333|2014|

In [50]:
df2.select((df2.arr_time / 60)).alias('duration_hrs').show()

+------------------+
|   (arr_time / 60)|
+------------------+
|15.583333333333334|
|25.083333333333332|
|27.533333333333335|
|             30.65|
|16.916666666666668|
|22.533333333333335|
|             17.35|
|              30.7|
|25.133333333333333|
|              39.2|
| 32.06666666666667|
|23.583333333333332|
|              17.3|
|3.6166666666666667|
|25.733333333333334|
|35.983333333333334|
|35.833333333333336|
|26.883333333333333|
|10.116666666666667|
|32.416666666666664|
+------------------+
only showing top 20 rows



In [51]:
df2.select(df2.origin, df2.dest ,(df2.arr_time / 60)).show()

+------+----+------------------+
|origin|dest|   (arr_time / 60)|
+------+----+------------------+
|   SEA| LAX|15.583333333333334|
|   SEA| HNL|25.083333333333332|
|   SEA| SFO|27.533333333333335|
|   PDX| SJC|             30.65|
|   SEA| BUR|16.916666666666668|
|   PDX| DEN|22.533333333333335|
|   PDX| OAK|             17.35|
|   SEA| SFO|              30.7|
|   SEA| SAN|25.133333333333333|
|   SEA| ORD|              39.2|
|   SEA| LAX| 32.06666666666667|
|   SEA| PHX|23.583333333333332|
|   SEA| LAS|              17.3|
|   SEA| ANC|3.6166666666666667|
|   SEA| SFO|25.733333333333334|
|   PDX| SFO|35.983333333333334|
|   SEA| SMF|35.833333333333336|
|   SEA| MDW|26.883333333333333|
|   SEA| BOS|10.116666666666667|
|   PDX| BUR|32.416666666666664|
+------+----+------------------+
only showing top 20 rows



In [52]:
df2.selectExpr('arr_time / 60 as duration_hours').show()

+------------------+
|    duration_hours|
+------------------+
|15.583333333333334|
|25.083333333333332|
|27.533333333333335|
|             30.65|
|16.916666666666668|
|22.533333333333335|
|             17.35|
|              30.7|
|25.133333333333333|
|              39.2|
| 32.06666666666667|
|23.583333333333332|
|              17.3|
|3.6166666666666667|
|25.733333333333334|
|35.983333333333334|
|35.833333333333336|
|26.883333333333333|
|10.116666666666667|
|32.416666666666664|
+------------------+
only showing top 20 rows



In [53]:
avg_speed = flights.select(flights.distance/ (flights.arr_time/60)).alias('avg_speed')

In [54]:
avg_speed.show(3)

+----------------------------+
|(distance / (arr_time / 60))|
+----------------------------+
|          61.219251336898395|
|          106.72425249169436|
|           24.66101694915254|
+----------------------------+
only showing top 3 rows



In [116]:
flights.groupBy().min('arr_time')

AnalysisException: "arr_time" is not a numeric column. Aggregation function can only be applied on a numeric column.

In [57]:
flights.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- arr_time_hrs: double (nullable = true)



In [58]:
flights.groupBy().min('arr_time_hrs').show()

+--------------------+
|   min(arr_time_hrs)|
+--------------------+
|0.016666666666666666|
+--------------------+



In [56]:
flights.groupby().max('arr_time_hrs').show()

+-----------------+
|max(arr_time_hrs)|
+-----------------+
|             40.0|
+-----------------+



In [59]:
flights.filter('carrier == "DL"').filter('origin == "SEA"').groupBy().avg('arr_time_hrs').show()

+-----------------+
|avg(arr_time_hrs)|
+-----------------+
|25.02641093474425|
+-----------------+



In [60]:
flights.show(2)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      arr_time_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|15.583333333333334|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|25.083333333333332|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
only showing top 2 rows



In [61]:
flights.filter('origin == "SEA"').filter('dest=="MIA"').groupby().sum('arr_time_hrs').show()

+------------------+
| sum(arr_time_hrs)|
+------------------+
|218.91666666666663|
+------------------+



In [62]:
flights.filter('origin == "SEA"').filter('dest=="JFK"').groupby().sum('arr_time_hrs').show()

+-----------------+
|sum(arr_time_hrs)|
+-----------------+
|3011.616666666668|
+-----------------+



In [63]:
flights.filter('origin == "SEA"').filter('dest=="LAX"').groupby().sum('arr_time_hrs').show()

+-----------------+
|sum(arr_time_hrs)|
+-----------------+
|11210.91666666667|
+-----------------+



In [64]:
flights.groupby('tailnum').count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [65]:
flights.groupby('tailnum').count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [66]:
flights.groupby('origin').avg('arr_time_hrs').show()

+------+------------------+
|origin| avg(arr_time_hrs)|
+------+------------------+
|   SEA|24.908754709498368|
|   PDX| 24.04415812894544|
+------+------------------+



In [118]:
flights.groupby('month', 'dest').avg('arr_delay').show()

+-----+----+-------------------+
|month|dest|     avg(arr_delay)|
+-----+----+-------------------+
|   11| TUS| -6.666666666666667|
|   11| ANC| -5.235294117647059|
|    1| BUR|               0.05|
|    1| PDX|-1.3846153846153846|
|    6| SBA|              -8.75|
|    5| LAX|-2.1842105263157894|
|   10| DTW|               -7.6|
|    6| SIT|                0.0|
|   10| DFW| 14.515151515151516|
|    3| FAI|              -12.0|
|   10| SEA|                0.4|
|    2| TUS|-23.666666666666668|
|   12| OGG|  22.09090909090909|
|    9| DFW|-0.5333333333333333|
|    5| EWR|                6.0|
|    3| RDM|               -4.0|
|    8| DCA|                4.4|
|    7| ATL| -8.162162162162161|
|    4| JFK| -7.923076923076923|
|   10| SNA|               -1.6|
+-----+----+-------------------+
only showing top 20 rows



In [69]:
import pyspark.sql.functions as F

In [121]:
airports.show(2)

+--------+--------------------+----------+-----------+-------+------+--------+
|"""faa""|            ""name""|   ""lat""|    ""lon""|""alt""|""tz""|""dst"""|
+--------+--------------------+----------+-----------+-------+------+--------+
|"""04G""|""Lansdowne Airpo...|41.1304722|-80.6195833|   1044|    -5|  ""A"""|
|"""06A""|""Moton Field Mun...|32.4605722|-85.6800278|    264|    -5|  ""A"""|
+--------+--------------------+----------+-----------+-------+------+--------+
only showing top 2 rows



In [130]:
airports2 = airports.withColumnRenamed('faa', 'dest')

In [131]:
airports2.show(2)

+--------+--------------------+----------+-----------+-------+------+--------+
|"""faa""|            ""name""|   ""lat""|    ""lon""|""alt""|""tz""|""dst"""|
+--------+--------------------+----------+-----------+-------+------+--------+
|"""04G""|""Lansdowne Airpo...|41.1304722|-80.6195833|   1044|    -5|  ""A"""|
|"""06A""|""Moton Field Mun...|32.4605722|-85.6800278|    264|    -5|  ""A"""|
+--------+--------------------+----------+-----------+-------+------+--------+
only showing top 2 rows



In [132]:
flights.join(airports2, on='dest', how='leftouter')

AnalysisException: USING column `dest` cannot be resolved on the right side of the join. The right-side columns: ["""faa"", ""name"", ""lat"", ""lon"", ""alt"", ""tz"", ""dst"""]

In [149]:
planes = spark.read.csv('../Datasets/planes.csv', header=True)

In [151]:
planes.show(2)

+------------+--------+--------------------+--------------------+------------+-----------+---------+---------+--------------+
|"""tailnum""|""year""|            ""type""|    ""manufacturer""|   ""model""|""engines""|""seats""|""speed""|   ""engine"""|
+------------+--------+--------------------+--------------------+------------+-----------+---------+---------+--------------+
| """N102UW""|    1998|""Fixed wing mult...|""AIRBUS INDUSTRIE""|""A320-214""|          2|      182|       NA|""Turbo-fan"""|
| """N103US""|    1999|""Fixed wing mult...|""AIRBUS INDUSTRIE""|""A320-214""|          2|      182|       NA|""Turbo-fan"""|
+------------+--------+--------------------+--------------------+------------+-----------+---------+---------+--------------+
only showing top 2 rows



In [144]:
planes = planes.withColumnRenamed("year", 'plane_year')

In [145]:
model_data= flights.join(planes, on='tailnum', how='leftouter')

AnalysisException: USING column `tailnum` cannot be resolved on the right side of the join. The right-side columns: ["""tailnum"", ""year"", ""type"", ""manufacturer"", ""model"", ""engines"", ""seats"", ""speed"", ""engine"""]